<a href="https://colab.research.google.com/github/terpene3/BeeSolver/blob/main/bee_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
import urllib.request
from zipfile import ZipFile
import pandas as pd
from io import BytesIO
import string
import math
import time
from collections import defaultdict
import altair as alt

In [137]:
# The chosen list contains some unicode characters for words with accents.
# For now we will ignored these.
# Tried these lists also but they aren't as high quality or don't have enough words:
  # "https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt"
  # "https://www.mit.edu/~ecprice/wordlist.10000"
uri="http://www.gwicks.net/textlists/usa2.zip"
resp = urllib.request.urlopen(uri).read()
words = ""
with ZipFile(BytesIO(resp)) as words_zip:
  binary_words = words_zip.open(words_zip.namelist()[0]).readlines()
raw_words = []
failed_words_count = 0
for w in binary_words:
  try:
    raw_words += [w.decode()]
  except:
    failed_words_count += 1
raw_words = [word.replace("\n", "") for word in raw_words]
raw_words = list(filter(lambda x: len(x) > 3, raw_words))
words = list(filter(lambda x: "\\x83" not in x, raw_words))
print("Failed to load words: ", failed_words_count)
print("Loaded words: ", len(words))

Failed to load words:  50
Loaded words:  76704


In [138]:
letters = "gliedaz"
required = "z"
if (len(letters) != 7):
  print("wrong letters length: " + str(len(letters)))
if (required not in letters):
  print("requires not in letters")
letters = set([c for c in letters])
optional_letters = letters - set([required])

In [139]:
%%timeit
found = []
for word in words:
  word_letters = set([c for c in word])
  if word_letters.issubset(letters) and required in word:
    found += [word]

10 loops, best of 3: 74.2 ms per loop


~100 ms per loop is almost fast enough to appear instant, but is it fast enough to solve the bee?

There are 26 * nCr(25,6) combinations

In [140]:
puzzles = 26 * math.factorial(25)/math.factorial(25-6)/math.factorial(6)
print(puzzles)

# This Takes 5 days:
puzzles * .1 /60 /60/ 24

4604600.0


5.329398148148148

We an preprocess the data to save on lookups. For example the puzzles abcdefg and abcdefh where a is the required letter both allow words that match abcdef. When deciding if a word is valid, the order of the optional letters in the puzzle doesn't matter and likewise the number of repititions is not valid.

We care about what are the unique letters in a word. The problem can be solved as finding all words that are composed solely of letters in each subsequence S where S = required letter + subsequence(optional_letters).

We can easily compute a mapping from a set of letters to words that are composed only of these letters. As order does not matter, the mapping can use a sorted unique characters that compose the corresponding words. As each word maps to 1 key, each word is stored only once in memory.

After preprocessing the problem can be solved by 2^6 - nCr(6,3) constant time map lookups. For simplicity we won't exclude subsequences with less than 4 char (nCr(6,3)) as this is a minor optimization.

A trie could be instead used to encode all words and searched; however, this is expected to be much slower as permutations instead of combinations of the input letters would need to be used in the search. A tree could instead be used where each node corresponds to words that are composed only of letters seen so far in traversing the tree such that the letter notes are "sorted." e.g. if one has seen node already in a DFS then no word containing "a" will be found while continuing deeper. This tree would have similar O complexity and memory requirements relative to the size of the world list used.

In [141]:
def SetToString(char_set):
  out = ""
  for char in char_set:
    out += char
  return WordToKey(out)

def WordToKey(word):
  return "".join(sorted(set([c for c in word])))

# Precompute the valid words for each key.
word_dict = defaultdict(lambda:[], key="some_value")
for word in words:
  key = WordToKey(word)
  word_dict[key] += [word]

In [142]:
def GetLetters(so_far, chars):
  if len(chars) == 0:
    return [so_far]
  return GetLetters(so_far + chars[0], chars[1:]) + GetLetters(so_far, chars[1:])

possible_chars = GetLetters("", SetToString(optional_letters))
possible_chars = [c + required for c in possible_chars]

In [143]:
%%timeit
out = []
for comb in possible_chars:
  key = WordToKey(comb)
  out += word_dict[key]

10000 loops, best of 3: 83.5 µs per loop


Now we have 1000x speedup with a brief preprocessing.

In [144]:
def GetLettersAllReq(so_far, chars):
  if len(so_far) == 6:
    return [so_far]
  if len(so_far) + len(chars) == 6:
    return [so_far + chars]
  return GetLettersAllReq(so_far + chars[0], chars[1:]) + GetLettersAllReq(so_far, chars[1:])

def GetAllBeeInputs():
  char_map = {}
  alpha = string.ascii_lowercase
  for i, char in enumerate(alpha):
    char_map[char] = GetLettersAllReq("",alpha[:i] + alpha[i+1:])
  return char_map

char_map = GetAllBeeInputs()

def GetCombinations(required_letter, optional_letters):
  possible_chars = GetLetters("", SetToString(optional_letters))
  possible_chars = [c + required_letter for c in possible_chars]
  return possible_chars


In [145]:
def SolveTheBee():
  solutions = {}
  start = time.monotonic()
  last_time = start
  for char, optional_letters in char_map.items():
    for opt in optional_letters:
      # Ignore if the solution with all letters is not present
      if len(word_dict[WordToKey(char + opt)]) == 0:
        continue
      solutions[(char,opt)] = []
      combs = GetCombinations(char, opt)
      for comb in combs:
        key = WordToKey(comb)
        solutions[(char,opt)] += word_dict[key]
    current_time = time.monotonic()
    print(char, "took", current_time - last_time, "s")
    last_time = current_time
  return solutions
solutions = SolveTheBee()
print("Total solutions:", len(solutions))

a took 1.3680853730002127 s
b took 0.6945639959994878 s
c took 0.8309251350001432 s
d took 0.8313515930003632 s
e took 1.3722070359999634 s
f took 0.5785150679994331 s
g took 0.7293848310000612 s
h took 0.7448167869997633 s
i took 1.1723174270000527 s
j took 0.41141770600006566 s
k took 0.5284855529998822 s
l took 0.9672131470006207 s
m took 0.7022894490000908 s
n took 1.040922698999566 s
o took 0.989912115000152 s
p took 0.6845078719998128 s
q took 0.41510714100058976 s
r took 1.0783364639992215 s
s took 1.1265977100001692 s
t took 1.1503238160003093 s
u took 0.9574872009998217 s
v took 0.5890467780000108 s
w took 0.5695565539999734 s
x took 0.48241861699989386 s
y took 0.703566148000391 s
z took 0.4814881679994869 s
Total solutions: 64491


In [146]:
%%timeit
solutions[required,(SetToString(optional_letters))]

The slowest run took 9.65 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.09 µs per loop


In [147]:
words_stored = 0
for solution in solutions.values():
  words_stored += len(words)

print("Now storing %d words or %.f times the orignal" % (words_stored, words_stored / len(words)))

Now storing 4946717664 words or 64491 times the orignal


Now we have a 10^5 speedup over our initial solution.

Example Solution

In [148]:
solution = solutions[required,(SetToString(optional_letters))]
rows = []
for word in solution:
  row = {}
  row["Word"] = word
  row["Length"] = len(word)
  row["Unique Letters"] = len(set([c for c in word]))
  rows += [row]
pd.set_option('display.max_rows', None)

pd.DataFrame(rows).sort_values(by=['Unique Letters'], ascending=False)

,Word,Length,Unique Letters
0,legalized,9,7
5,idealized,9,6
20,elegized,8,6
1,zigzagged,9,6
11,legalize,8,6
4,idealize,8,6
2,glazed,6,6
3,gazed,5,5
6,dazzle,6,5
13,glaze,5,5


In [149]:
rows = []
for key, val in solutions.items():
  row = {}
  row["Letter"] = key[0]
  row["Letters in Puzzle"] = key[1]
  row["Words"] = len(set(val))
  if (len(set(val)) > 0):
    for length in range(4,15):
      row["Words of length " + str(length)] = sum([len(word) == length for word in val])
  rows += [row]
solution_frame = pd.DataFrame(rows)

In [150]:
alt.Chart(solution_frame.groupby("Letter", as_index=False).sum()).mark_bar().encode(x="Letter:O", y="Words:Q"
).interactive()

alt.Chart(...)

In [151]:
alt.Chart(solution_frame.groupby("Words", as_index=False).count()).mark_bar().\
  encode(x="Words:Q", y="Letter", tooltip=["Words", "Letter"]).interactive()

alt.Chart(...)

In [152]:
alt.Chart(solution_frame.groupby("Letter", as_index=False).mean()).\
  mark_bar().encode(x="Letter:O", y=alt.Y("Words:Q", title="Mean Words in Solution"))

alt.Chart(...)

In [153]:
alt.Chart(solution_frame.groupby("Letter", as_index=False).count()).\
  mark_bar().encode(x="Letter:O", y=alt.Y("Words:Q", title="Total Words in Solutions"))

alt.Chart(...)

In [154]:
alt.Chart(solution_frame.groupby("Letter", as_index=False).count()).\
  mark_bar().encode(x="Letter:O", y=alt.Y("Words:Q", title="Total Solutions"), tooltip=["Words", "Letter"]).interactive()

alt.Chart(...)

Getting Examples that have lots of long words


In [155]:
s="Words of length 10"
solution_frame[solution_frame[s] > 15]

,Letter,Letters in Puzzle,Words,Words of length 4,Words of length 5,Words of length 6,Words of length 7,Words of length 8,Words of length 9,Words of length 10,Words of length 11,Words of length 12,Words of length 13,Words of length 14
3066,a,eginrt,311,43,39,66,75,31,28,16,9,3,1,0
3507,a,einrst,475,60,78,101,100,73,37,16,7,3,0,0
4088,a,ginort,187,37,29,27,30,22,22,16,3,1,0,0
14210,e,ainrst,593,66,103,130,129,90,47,19,7,2,0,0
16696,e,doprsu,331,62,69,75,55,23,24,16,5,2,0,0
17585,e,imnrst,418,60,87,87,78,52,33,16,4,1,0,0
17650,e,inprst,495,62,92,104,99,69,41,16,9,3,0,0
17681,e,ioprst,468,60,82,102,104,61,31,16,9,3,0,0
19656,g,aeinrt,270,24,31,54,68,35,29,17,8,3,1,0
24465,i,aegnrt,287,27,36,51,74,40,28,17,10,3,1,0
